# 제2 고지 : 자연스러운  코드로 
## STEP 21 : 연산자 오버로드(2)

이전 단계에서 `+`,`*` 연산자를 정의함으로서 더 손쉬운 사용이 가능해졌다.  
하지만  `a * np.array(2.0)` 처럼 `ndarray` 인스턴스와는 사용할 수 없다. 또한 `3+b` 와 같이 수치 데이터와도 사용할 수 없다.  
그래서 이번 단계에서는 `Variable` 인스턴스와 `ndarray`인스턴스, 그리고 `int`/`float` 등의 수치 데이터도 함께 사용할 수 있도록 개선해본다. 

### 21.1 ndarray와 함께 사용하기 

우선 `Variable` 을 `ndarray` 인스턴스와  함께 사용할 수 있도록 한다. 즉, `a * np.array(2.0)` 이라는 코드를 실행하면 **`ndarray` 인 인스턴스를 자동으로 `Variable`인스턴스로 변환**한다.  
이를 구현하기 위해 다음 과정을 따른다.

1. `as_variable` 함수를 구현한다.
2. `Function` 클래스의 `__call__()` 메서드가 `as_variable` 을 이용하도록 구현한다.

In [15]:
import weakref
import numpy as np 


###############################
def as_variable(obj):
    if isinstance(obj,Variable):
        return obj 
    return Variable(obj)
###############################


class Variable:
    def __init__(self, data,name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name  # 변수 구분을 위한 `이름` 설정
        self.grad = None
        self.creator = None
        self.generation = 0
        
    @property
    def ndim(self):
        return self.data.ndim
    @property
    def shape(self):
        return self.data.shape
    @property
    def size(self):
        return self.data.size
    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return "variable(None)"
        p = str(self.data).replace("\n","\n"+" "*9)
        return 'variable('+p+')'
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None
        
    def backward(self,retain_grad=False):  # `retain_grad` 추가 
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref이기 때문에 y()로 호출
               
    ######################################
    def __mul__(self,other):
        return mul(self,other)
    def __add__(self,other):
        return add(self,other)
    ######################################
    

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x



class Function:
    def __call__(self, *inputs):
        ################################
        inputs = [as_variable(x) for x in inputs]
        ################################
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

class Add(Function):
    def forward(self, x0,x1):
        y = x0 + x1
        return y
    def backward(self, gy):
        # 역전파시 , 입력이 1개 , 출력이 2개 
        return gy,gy 
    
def add(x0,x1):
    return Add()(x0,x1)

class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0
    
def mul(x0,x1):
    return Mul()(x0,x1)

In [16]:
x = Variable(np.array(2.0))
y = x+np.array(3.0)
print(y)

variable(5.0)


### 21.2 float,int와 함께 사용하기 

이어서 `float` 와 `int`, 그리고 `np.float64`,`np.int64` 같은 타입과도 함께 사용할 수 있도록 한다. 즉, `x +3.0` 와 같은 코드를 실행할 수 있도록 한다.  
가장 간단한 방법은 `as_array()` 함수를 이용해 **우측항이 수치형 데이터인 경우 `ndarray` 인스턴스로 변환**하는 것이다. 

In [21]:
def add(x0,x1):
    x1 = as_array(x1)
    return Add()(x0,x1)

def mul(x0,x1):
    x1= as_array(x1)
    return Mul()(x0,x1)


In [22]:
x = Variable(np.array(2.0))
y = x+3.0
print(y)

variable(5.0)


### 21.3 문제점 1 : 첫 번째 인수가 float나 int인 경우 

앞서 해결한 방식은 **두가지 문제**가 남아있는데, 첫번째 문제에 대해 살펴보자. 


현재 `x * 2.0` 코드를 제대로 실행할 수 있지만, `2.0 * x` 를 실행하면 오류가 난다.  
그 이유는 다음과 같다. 

1. 연산자 왼쪽의 `2.0` 의 `__mul__` 를 호출 시도한다. 
2. 수치형 데이터의 `__mul__` 가 구현되지 않았으므로 `*` 연산자 오른쪽에 있는 `x`의 특수 메서드를 호출 시도한다.
3. `x`가 오른쪽에 있기 때문에 `__mul__` 메서드가 아닌 `__rmul__`를 호출 시도한다.
4. 하지만 `__rmul__` 이 구현되어 있지 않으므로, 오류가 발생한다.

여기서 주목할 것은 **이항연산자의 경우 피연산자의 위치에 따라 호출되는 메서드가 다르다**는 것이다.  

![image](../assets/%EA%B7%B8%EB%A6%BC%2021-1.png)


따라서 이번 문제를 해결하기 위해서 `__rmul__` 를 호출한다. 여기서 주목할 것은 **곱셈, 덧셈의 연산 결과는 순서를 구분할 필요가 없으므로** 다음과 같이 간단하게 구현가능하다.



In [23]:
Variable.__add__=add 
Variable.__radd__=add 
Variable.__mul__=mul
Variable.__rmul__=mul

In [24]:
x = Variable(np.array(2.0))
y = 3.0*x + 1.0
print(y)

variable(7.0)


### 21.4 문제점 2 : 좌항이 ndarray 인스턴스인 경우

마지막 남은 문제는 `ndarray` 가 인스턴스 좌항이고 `Variable` 인스턴스가 우항인 경우이다. 
예를들어, 다음과 같은 경우이다. 

```python
x = Variable(np.array([1.0]))
y = np.array([2.0]) + x 
```

이 코드는 `ndarray` 인스턴스의 `__add__` 호출 한다. 하지만 우리는 우항의 `Variable` 인스턴스의 `__radd__`가 호출되기를 원한다.  
즉, 여러 피연산자의 연산자 메서드가 정의되어 있을 때 **연산자 우선순위**를 정해줘야 한다는 것이다.

구체적으로 `Variable` 인스턴스의 속성에 `__array_priority__` 를 추가하여 우선순위를 높여준다.  
참고로, `__array_priority__` 속성은 `ndarray` 의 속성으로 `numpy` 내부적으로 우선순위를 정하는 속성값이다.

In [29]:
class Variable:
    #################################
    # TODO: 주석 처리를 통해 결과를 비교해보자. 
    __array_priority__ = 200 
    #################################
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError("{} is not supported".format(type(data)))

        self.data = data
        self.name = name  # 변수 구분을 위한 `이름` 설정
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def shape(self):
        return self.data.shape

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return "variable(None)"
        p = str(self.data).replace("\n", "\n" + " " * 9)
        return "variable(" + p + ")"

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):  # `retain_grad` 추가
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref이기 때문에 y()로 호출

    def __mul__(self, other):
        return mul(self, other)

    def __rmul__(self, other):
        return mul(self, other)

    def __add__(self, other):
        return add(self, other)

    def __radd__(self, other):
        return add(self, other)

In [30]:
x = Variable(np.array([1.0]))
y = np.array([2.0]) + x 
print(y)

variable([3.])
